<a href="https://colab.research.google.com/github/NikSh1891/NikSh1891/blob/main/Food_delivery_company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Продуктовая аналитика в компании по доставке продуктов питания на дом.
Сервис доступен приложении на ios и на android. Настроена фронтовая аналитика в AppMetrica, и в конце квартала маркетинг-менеджер поставил задачу проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения.\
Выгружены данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np

Загружаем данные

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1. Анализ данных/Collab/Product_analysis_lesson_3/KC_case_data .csv')

## Предварительный анализ данных

In [ ]:
# Посмотрим на размер датасета и на его содержание
print(df.shape)
df.sample(5)

(2747968, 8)


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
1095331,2020-02-07,search,NaN,ios,8340851,female,Saint-Petersburg,instagram_ads
2649523,2020-03-14,app_install,NaN,android,29208510,female,Moscow,yandex-direct
1982112,2020-03-09,purchase,636.0,ios,1570840,male,Moscow,instagram_ads
71467,2020-01-05,app_start,NaN,android,21573876,male,Moscow,vk_ads
847096,2020-01-29,search,NaN,ios,30202436,female,Moscow,referal


In [ ]:
# Выведем представленные в таблице виды данных
df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [ ]:
# Проверим данные на наличие пропусков
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

Наличие пропусков в столбце с суммой покупки является следствием того что не все пользователи совершали покупки. Это можно проверить вычислив кол-во подтверждений покупки в столбце с событиями и вычесть получившееся число из общего размера датасета. Должно получиться число равное кол-ву пропусков в столбце с суммой покупки.

In [ ]:
# Вычитаем из общего числа строк кол-во подтверждений покупки в столбце с событиями
df.shape[0] - df[df.event == 'purchase'].shape[0]

2606585

## MAU февраля
MAU(Monthly Active Users) - кол-во уникальных пользователей которые зашли в приложение за месяц.

In [ ]:
# Вычислим кол-во уникальных пользователей зашедших в приложение за месяц февраль
print('MAU февраля: ', df.query("'2020-02-01' <= date <= '2020-02-29'").device_id.nunique())

MAU февраля:  75032


## Количество установок января
Вычислим кол-во установок приложения за месяц январь

In [ ]:
# Отфильтруем данные по месяцу январю и событию установка и посчитаем колзво уникальных пользователей
print('Кол-во установок января: ', df.query("'2020-01-01' <= date <= '2020-01-31' & event == 'app_install'").device_id.nunique())

Кол-во установок января:  80297


## Когортный анализ
Проведем когортный анализ для  ответа на вопрос: Для какой когорты конверсия в покупку была наибольшей?\
Присвоим пользователям когорты по дню установки приложения и посчитаем для них конверсию из установки в покупку в течение 7ми дней.

In [ ]:
# Преобразуем столбец date из типа object в тип datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Сформируем два датасета с уникальными ID пользователей с инсталлами и с покупками
install = df.query("event == 'app_install'")[['device_id', 'date']].drop_duplicates('device_id')
purchase = df.query("event == 'purchase'")[['device_id', 'date']].drop_duplicates('device_id')

In [ ]:
# Сформируем датасет с общим кол-вом пользователей в когортах
kog_amount = install.groupby('date', as_index=False).agg({'device_id': 'count'})
kog_amount.head()

,date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [ ]:
install = install.rename(columns={'date': 'install'})
purchase = purchase.rename(columns={'date': 'purchase'})

In [ ]:
# Объединим установки и покупки по столбцу с ID с помощью left merge
m_data = install.merge(purchase, how='left', on='device_id')
m_data.head()

,device_id,install,purchase
0,4921563,2020-01-01,2020-01-10
1,1311583,2020-01-01,NaT
2,23006325,2020-01-01,2020-01-01
3,5645652,2020-01-01,NaT
4,8798567,2020-01-01,NaT


In [ ]:
# Создадим новый столбец с разницой между датой покупки и датой установки
m_data['dif'] = m_data.purchase - m_data.install
m_data.head()

,device_id,install,purchase,dif
0,4921563,2020-01-01,2020-01-10,9 days
1,1311583,2020-01-01,NaT,NaT
2,23006325,2020-01-01,2020-01-01,0 days
3,5645652,2020-01-01,NaT,NaT
4,8798567,2020-01-01,NaT,NaT


Будем считать пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [ ]:
# Создадим столбец с данными сконвертировался пользователь 'y' или не сконвертировался 'n'
day_const = pd.Timedelta(days=7)
m_data['convert_user'] = m_data['dif'].apply(lambda x: 'y' if x <= day_const else 'n')
m_data.head()

,device_id,install,purchase,dif,convert_user
0,4921563,2020-01-01,2020-01-10,9 days,n
1,1311583,2020-01-01,NaT,NaT,n
2,23006325,2020-01-01,2020-01-01,0 days,y
3,5645652,2020-01-01,NaT,NaT,n
4,8798567,2020-01-01,NaT,NaT,n


In [ ]:
# Отфильтруем только сконвертировавшихся пользователей, сгруппируем данные по дате установки и посчитаем кол-во уникальных пользователей
conv_user = m_data.query("convert_user == 'y'").groupby('install', as_index=False).agg({'device_id': 'count'}).rename(columns={'install': 'date', 'device_id': 'conv_users'})
conv_user.head()

,date,conv_users
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587


In [ ]:
# Объединим данные о сконвертировавшихся пользователях с данными об общем кол-ве пользователей в когортах для возможности подсчета конверсии
final_data = conv_user.merge(kog_amount, on='date')

In [ ]:
# Рассчитаем конверсию
final_data['retention_rate'] = (final_data.conv_users / final_data.device_id * 100).round(2)
final_data.head()

,date,conv_users,device_id,retention_rate
0,2020-01-01,1408,3579,39.34
1,2020-01-02,1186,3144,37.72
2,2020-01-03,834,2402,34.72
3,2020-01-04,639,1831,34.90
4,2020-01-05,587,1671,35.13


#### Выводы по когортному анализу:
- Наибольшая конверсия была у когорты с датой 01.01.2020
- CR когорты 01.01.2020 - 39,34 %

## Анализ платных маркетинговых каналов
С какого платного маркетингового канала пришло больше всего новых пользователей?\
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы.

In [ ]:
# Сгруппируем по видам маркетинговых каналов, посчитаем кол-во уникальных пользователей пришедших с каждого из каналов
df.groupby('utm_source', as_index=False).agg({'device_id': 'nunique'}).sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


#### Вывод по анализу маркетинговх каналов:
- С Яндекс-директа приходит больше всего новых пользователей

## Анализ воронки продаж
На каком шаге отваливается больше всего зарегистрированных пользователей?

In [ ]:
# Сфомируем датасет с пользователями которые проходили регистрацию
reg_date_data = df[df['event'] == 'register'][['device_id', 'date']].rename(columns={'date': 'reg_date'})

In [ ]:
# Объединим исходный датасет с reg_date_data
reg_data = df.merge(reg_date_data, how='left', on='device_id')
reg_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,reg_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT


In [ ]:
# Отфильтруем только зарегистрированных пользователей
reg_users = reg_data.query("reg_date < date")
reg_users.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,reg_date
15148,2020-01-02,app_start,NaN,ios,3163406,female,Moscow,instagram_ads,2020-01-01
15175,2020-01-02,app_start,NaN,ios,2169108,female,Moscow,google_ads,2020-01-01
15197,2020-01-02,app_start,NaN,android,4863305,male,Saint-Petersburg,vk_ads,2020-01-01
15210,2020-01-02,app_start,NaN,android,15248490,male,Moscow,facebook_ads,2020-01-01
15214,2020-01-02,app_start,NaN,ios,25617659,male,Moscow,vk_ads,2020-01-01


In [ ]:
# Сформируем датасет с видом события пользователя
# Сгруппируем по событию, посчитаем кол-во уникальных пользователей по каждому событию
event_data = reg_users.groupby('event', as_index=False).agg({'device_id': 'nunique'}).sort_values('device_id', ascending=False)
# Добавим столбец со смещением
event_data['shift'] = event_data['device_id'].shift()
# Посчитаем конверсию из предыдущего в следующее событие
event_data['v_conv'] = (event_data['device_id'] / event_data['shift'] * 100).round(2)
event_data

,event,device_id,shift,v_conv
0,app_start,40991,NaN,NaN
3,search,40482,40991.0,98.76
1,choose_item,37926,40482.0,93.69
4,tap_basket,34517,37926.0,91.01
2,purchase,24880,34517.0,72.08


#### Вывод по анализу воронки продаж:
- Больше всего пользователей отваливается на этапе перехода из корзины в подтвеждение покупки

## Анализ конверсии маркетинговых каналов
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [ ]:
# Сформируем данные с кол-вом уникальных пользователей совершивших покупку с группировкой по маркетинговым каналам
purchase = df.query("event == 'purchase'").groupby('utm_source', as_index=False).agg({'device_id': 'nunique'}).rename(columns={'device_id': 'pur_count'})
purchase

,utm_source,pur_count
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [ ]:
# Сформируем данные с кол-вом уникальных пользователей впервые открывших приложение с группировкой по маркетинговым каналам
app_start = df.query("event == 'app_start'").groupby('utm_source', as_index=False).agg({'device_id': 'nunique'}).rename(columns={'device_id': 'app_start_count'})
app_start

,utm_source,app_start_count
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924
5,vk_ads,34079
6,yandex-direct,40707


In [ ]:
# Объединим данные о маркетинговых каналах с пользователями совершившими покупку и пользователями впервые открывшие приложение
final_data = purchase.merge(app_start, how='left', on='utm_source')
# Создадим столбец и вычислим конверсию по каждому маркетинговому каналу
final_data['CR'] = (final_data.pur_count / final_data.app_start_count * 100).round(2)
final_data.sort_values('CR')

,utm_source,pur_count,app_start_count,CR
6,yandex-direct,12028,40707,29.55
2,google_ads,11339,38092,29.77
0,-,16598,52268,31.76
3,instagram_ads,10762,31045,34.67
1,facebook_ads,9017,25957,34.74
5,vk_ads,12364,34079,36.28
4,referal,6362,15924,39.95


#### Выводы по анализу конверсии маркетинговых каналов:
- Яндекс-директ показал самую низкую конверсию в первую покупку среди других платных каналов

## Анализ медианного чека по маркетинговым каналам
Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [ ]:
# Отфильтруем данные только с первыми покупками, группируем по каналу продаж и посчитаем медианный чек по каждому каналу
df.query("event == 'purchase'").drop_duplicates('device_id').groupby('utm_source', as_index=False).agg({'purchase_sum': 'median'}).sort_values('purchase_sum', ascending=False)

,utm_source,purchase_sum
0,-,398.5
4,referal,395.5
3,instagram_ads,393.5
5,vk_ads,393.0
6,yandex-direct,392.5
2,google_ads,390.5
1,facebook_ads,389.0


#### Выводы анализа медианного чека по маркетинговым каналам:
- Реферальная программа приносит пользователей с самым высоким медианным чеком

## Анализ ROMI маркетинговых каналов
Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?\
ROMI (Return on Marketing Investment) — коэффициент возврата маркетинговых инвестиций.

Имеются данные по затратам на рекланые каналы:\
Яндекс – 10 491 707 руб.\
Гугл – 10 534 878 руб.\
Фейсбук – 8 590 498 руб.\
Инстаграм – 8 561626 руб.\
ВК – 9 553 531руб.

In [ ]:
# Сгруппируем данные по каналу продаж и посчитатем сумму покупок по каждому каналу
df_ad = df.groupby('utm_source', as_index=False).agg({'purchase_sum': 'sum'})
df_ad

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [ ]:
df_ad['ad_cost'] = [0, 8590498, 10534878, 8561626, 0, 9553531, 10491707]

In [ ]:
df_ad['ROMI'] = (df_ad['purchase_sum'] - df_ad['ad_cost']) / df_ad['ad_cost']

In [ ]:
df_ad.sort_values('ROMI', ascending=False)

,utm_source,purchase_sum,ad_cost,ROMI
0,-,21449749.5,0,inf
4,referal,8837044.5,0,inf
5,vk_ads,16389652.5,9553531,0.715560
3,instagram_ads,14546969.0,8561626,0.699090
1,facebook_ads,12249901.0,8590498,0.425983
6,yandex-direct,13915368.0,10491707,0.326321
2,google_ads,12868276.0,10534878,0.221493


#### Вывод по анализу ROMI:
- Cреди платных каналов привлечения самый высокий ROMI имеет канал ВК.